In [3]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset, load_dataset, DatasetDict
from transformers import PatchTSTForClassification, PatchTSTConfig, set_seed
from tsfm_public.toolkit.dataset import ClassificationDFDataset, ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index

set_seed(42)

d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:496: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:353: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:353: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Program\miniconda3\envs\DL\lib\site-packages\transformers\utils\generic.py:353: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_p

In [10]:
overall_df = pd.read_csv("../data/overall_20090105-20250411.csv", parse_dates=["date"])
overall_df = overall_df.bfill().ffill()
train_df, eval_df = overall_df.iloc[:-500], overall_df.iloc[-500:]

In [11]:
context_length = 256
patch_length = 8
timestamp_column = "date"
target_columns = ["EPValue", "10年"]
conditional_columns = ["open", "close"]

In [12]:
time_series_preprocessor = TimeSeriesPreprocessor(
    timestamp_column=timestamp_column,
    target_columns=target_columns,
    # conditional_columns=conditional_columns,
    context_length=context_length,
    scaling=True,
)
time_series_preprocessor = time_series_preprocessor.train(train_df)

In [81]:
train_ds = ForecastDFDataset(
    time_series_preprocessor.preprocess(train_df),
    timestamp_column=timestamp_column,
    target_columns=target_columns,
    conditional_columns=conditional_columns,
    context_length=context_length,
    prediction_length=0,
    autoregressive_modeling=True,
)

In [13]:
train_ds_cls = ClassificationDFDataset(
    time_series_preprocessor.preprocess(train_df),
    timestamp_column=timestamp_column,
    input_columns=target_columns + conditional_columns,
    context_length=context_length,
    label_column="close",
)
eval_ds_cls = ClassificationDFDataset(
    time_series_preprocessor.preprocess(eval_df),
    timestamp_column=timestamp_column,
    input_columns=target_columns + conditional_columns,
    context_length=context_length,
    label_column="close",
)

In [14]:
train_ds_new = Dataset.from_list(train_ds_cls)
eval_ds_new = Dataset.from_list(eval_ds_cls)
overall_ds = DatasetDict({"train": train_ds_new, "test": eval_ds_new})
overall_ds.set_format("np")

In [15]:
def process(example):
    old_past_values = example.pop("past_values")
    open_price = old_past_values[:, :, 2]
    close_price = old_past_values[:, :, 3]
    prompt_ids = old_past_values[:, :, :2]
    prompt_mask = example.pop("past_observed_mask")[:, :, :2]

    return {"prompt_ids": prompt_ids, "prompt_mask": prompt_mask, "open_price": open_price, "close_price": close_price}

In [16]:
overall_ds = overall_ds.map(process,
                            remove_columns=["target_values", "id", "past_values", "past_observed_mask"],
                            batched=True)

Map:   0%|          | 0/3197 [00:00<?, ? examples/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

In [17]:
overall_ds["train"].to_parquet("../data/train.parquet")
overall_ds["test"].to_parquet("../data/eval.parquet")

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1526840

In [4]:
train_ds = load_dataset("parquet", data_files="../data/train.parquet", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
train_ds.set_format("np")

In [19]:
overall_ds["train"][0]["prompt_mask"].shape

(256, 2)

In [20]:
x = torch.tensor([[2, 4, 6, 5, 9, 7, 3, 8, 1, 0], [0, 2, 1, 9, 4, 5, 6, 8, 3, 7], [2, 5, 8, 9, 6, 3, 0, 1, 4, 7],
                  [5, 2, 1, 6, 8, 4, 9, 0, 3, 7]])

In [ ]:
x.repeat_interleave(3, dim=0)

In [21]:
torch.repeat_interleave(x, repeats=3, dim=0)

tensor([[2, 4, 6, 5, 9, 7, 3, 8, 1, 0],
        [2, 4, 6, 5, 9, 7, 3, 8, 1, 0],
        [2, 4, 6, 5, 9, 7, 3, 8, 1, 0],
        [0, 2, 1, 9, 4, 5, 6, 8, 3, 7],
        [0, 2, 1, 9, 4, 5, 6, 8, 3, 7],
        [0, 2, 1, 9, 4, 5, 6, 8, 3, 7],
        [2, 5, 8, 9, 6, 3, 0, 1, 4, 7],
        [2, 5, 8, 9, 6, 3, 0, 1, 4, 7],
        [2, 5, 8, 9, 6, 3, 0, 1, 4, 7],
        [5, 2, 1, 6, 8, 4, 9, 0, 3, 7],
        [5, 2, 1, 6, 8, 4, 9, 0, 3, 7],
        [5, 2, 1, 6, 8, 4, 9, 0, 3, 7]])

In [93]:
result = torch.tensor([train_ds_new[0]["past_values"]]).unfold(1, 32, 1).permute(0, 1, 3, 2)

In [108]:
result.flatten(end_dim=1).shape

torch.Size([225, 32, 2])

In [25]:
res = x.unfold(1, 5, 1)

In [29]:
res.shape

torch.Size([4, 6, 5])

In [51]:
config = PatchTSTConfig(
    num_input_channels=4,
    context_length=32,
    patch_length=patch_length,
    patch_stride=patch_length,
    random_mask_ratio=0.4,
    d_model=128,
    num_attention_heads=16,
    num_hidden_layers=3,
    ffn_dim=256,
    dropout=0.2,
    head_dropout=0.2,
    pooling_type="mean",
    channel_attention=False,
    scaling="std",
    loss="mse",
    pre_norm=True,
    norm_type="batchnorm",
    num_targets=2,
)
model = PatchTSTForClassification(config).to("cuda")


In [52]:
model

PatchTSTForClassification(
  (model): PatchTSTModel(
    (scaler): PatchTSTScaler(
      (scaler): PatchTSTStdScaler()
    )
    (patchifier): PatchTSTPatchify()
    (masking): Identity()
    (encoder): PatchTSTEncoder(
      (embedder): PatchTSTEmbedding(
        (input_embedding): Linear(in_features=8, out_features=128, bias=True)
      )
      (positional_encoder): PatchTSTPositionalEncoding(
        (positional_dropout): Identity()
      )
      (layers): ModuleList(
        (0-2): 3 x PatchTSTEncoderLayer(
          (self_attn): PatchTSTAttention(
            (k_proj): Linear(in_features=128, out_features=128, bias=True)
            (v_proj): Linear(in_features=128, out_features=128, bias=True)
            (q_proj): Linear(in_features=128, out_features=128, bias=True)
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
          (dropout_path1): Identity()
          (norm_sublayer1): PatchTSTBatchNorm(
            (batchnorm): BatchNorm1d(128, 